# 8 Puzzle solver
In this repository we're going to solve this puzzle using $ A^* $ and $ IDA $

#### imports:

In [12]:
import pandas as pandas
import numpy as np

#### Test case 1:

In [19]:
input_puzzle = [
    [1, 2, 3],
    [4, 0, 5],
    [7, 8, 6]
]

print('Input: ')
print(pandas.DataFrame(input_puzzle))
print()

desired_output = [
    [1, 2, 3],
    [4, 5, 0],
    [7, 8, 6]
]

print('Desired Output:')
print(pandas.DataFrame(desired_output))

Input: 
   0  1  2
0  1  2  3
1  4  0  5
2  7  8  6

Desired Output:
   0  1  2
0  1  2  3
1  4  5  0
2  7  8  6


### The node class:

In [ ]:
class Node:

    def __init__(self, parent, g, ):
